In [1]:
#QRM gate counts
#June 27th 2023
import math
import numpy as np
from qrm_counts import *
r, m = 7, 10

print(naive_CX_count(r, m))
print(std_CX_count(r, m))
print(rec_CX_count(r, m))

param_list = [(1, 3), (2, 4), (3, 4), (2, 5), (3, 5), (4, 5), (5, 5), (3, 6), (4, 6), (5, 6), (3, 7), (4, 7), (5, 7), (6, 7), (4, 8), (5, 8), (6, 8), (7, 8), (8, 10), (9, 10)]

print(rec_CX_count_assym(2, 5, 2, 5))

for params in param_list:
    r, m = params
    print('\n', r, m)
    print(naive_CX_count(r, m))
    print(std_CX_count(r, m))
    print(rec_CX_count(r, m))

57880
20664
5096
66

 1 3
16
12
10

 2 4
61
53
30

 3 4
65
29
32

 2 5
176
112
66

 3 5
206
150
78

 4 5
211
61
80

 5 5
211
0
80

 3 6
614
470
176

 4 6
659
367
190

 5 6
665
125
192

 3 7
1744
960
372

 4 7
1989
1389
430

 5 7
2052
840
446

 6 7
2059
253
448

 4 8
5821
3933
930

 5 8
6213
3605
1004

 6 8
6297
1857
1022

 7 8
6305
509
1024

 8 10
58015
8627
5118

 9 10
58025
2045
5120


In [1]:
import stim
circuit = stim.Circuit()

# First, the circuit will initialize a Bell pair.
circuit.append("H", [0])
circuit.append("CNOT", [0, 1, 3, 4])

# Then, the circuit will measure both qubits of the Bell pair in the Z basis.
circuit.append("M", [0, 1])

# Let's see the circuit's representation using stim's circuit language:
print(repr(circuit))



circuit = stim_CNOT_list(circuit, [0, 1], [2, 3])
circuit = stim_H_list(circuit, [0, 1])
print(circuit)

stim.Circuit('''
    H 0
    CX 0 1 3 4
    M 0 1
''')


NameError: name 'stim_CNOT_list' is not defined

In [25]:
from qrm_circuits import QRM_rec_circuit, tequila_to_stim

circ, info = QRM_rec_circuit(3, 4)

st_circuit= tequila_to_stim(circ)
st_circuit.append("DEPOLARIZE2", [0, 1, 2, 3], 0.5)

In [12]:
from qrm_circuits import QRM_rec_circuit, get_qubit_partition
from qrm_utils import get_qiskit_circuit

c, i = QRM_rec_circuit(3, 4, partitions=[2, [2], [2]], only_cnots=True)
print(get_qiskit_circuit(c))

                                                                            »
 q_0: ──■──────────────■────────────────────────■────────────────────────■──»
        │              │                        │                      ┌─┴─┐»
 q_1: ──┼────■─────────┼────■───────────────────┼───────────────────■──┤ X ├»
        │    │       ┌─┴─┐  │                   │                   │  └───┘»
 q_2: ──┼────┼────■──┤ X ├──┼───────────────────┼────■──────────────┼───────»
        │    │    │  └───┘┌─┴─┐                 │    │              │       »
 q_3: ──┼────┼────┼────■──┤ X ├─────────────────┼────┼──────────────┼────■──»
      ┌─┴─┐  │    │    │  └───┘                 │    │              │    │  »
 q_4: ┤ X ├──┼────┼────┼────■─────────■─────────┼────┼─────────■────┼────┼──»
      └───┘┌─┴─┐  │    │    │         │         │    │       ┌─┴─┐  │    │  »
 q_5: ─────┤ X ├──┼────┼────┼────■────┼─────────┼────┼────■──┤ X ├──┼────┼──»
           └───┘┌─┴─┐  │  ┌─┴─┐  │    │         │    │    │  └──

In [6]:
from qrm_counts import rec_CX_count_assym_punc
from qrm_circuits import QRM_rec_assym_punc_circuit
param_list = [(1, 3), (1, 4), (2, 5), (2, 6), (2, 7)]
for param in param_list:
    r, m = param
    a = rec_CX_count_assym_punc(r, m, r, m, state_prep=True)
    b = rec_CX_count_assym_punc(r, m, r, m, state_prep=False)

    #check with circuits
    c1, i1 = QRM_rec_assym_punc_circuit(r, m, r, m, state_prep=True, only_cnots=False)
    c2, i2 = QRM_rec_assym_punc_circuit(r, m, r, m, state_prep=False, only_cnots=False)
    
    '''if a != len(c1.gates):
        print('Somethings off')
    if b != len(c2.gates):
        print('Somethings off')'''
    
    print('r: {} m: {}, |0>: {}, D:{} |+>: {}, D:{}'.format(r, m, a, c1.depth, b, c2.depth))

r: 1 m: 3, |0>: 8, D:4 |+>: 9, D:5
r: 1 m: 4, |0>: 22, D:5 |+>: 24, D:6
r: 2 m: 5, |0>: 63, D:6 |+>: 65, D:10
r: 2 m: 6, |0>: 150, D:7 |+>: 153, D:11
r: 2 m: 7, |0>: 332, D:8 |+>: 336, D:12


In [1]:
from qrm_circuits import QRM_rec_punc_circuit, QRM_punc_std_circuit, canonical_CSS
from qrm_utils import get_qiskit_circuit, check_if_same_circuit
from qrm_matrices import get_QRM_punc_generator
import tequila
from qrm_counts import rec_CX_count_punc

r, m = 2, 3
c, i = QRM_rec_punc_circuit(r, m, only_cnots=True)
print(len(c.gates))
print(rec_CX_count_punc(r, m))
#c2, i2 = QRM_punc_std_circuit(r, m)

G = get_QRM_punc_generator((r, m), (r, m))
c2, i = canonical_CSS(G, (2**m)-1)

print(get_qiskit_circuit(c), get_qiskit_circuit(c2))
tequila.simulate(c)
print(check_if_same_circuit(c, c2))

[0, 1, 2] [3, 4, 5, 6]
[0] [1, 2]
Invalid parameters r1: 1, r2: 0, m: 1
9
11
               ┌───┐                    
q_0: ──■───────┤ X ├────────────────────
       │       └─┬─┘                    
q_1: ──┼────■────┼────■─────────────────
       │    │    │  ┌─┴─┐┌───┐          
q_2: ──┼────┼────■──┤ X ├┤ X ├──────────
       │    │       └───┘└─┬─┘          
q_3: ──┼────┼────■─────────┼─────────■──
     ┌─┴─┐  │    │         │       ┌─┴─┐
q_4: ┤ X ├──┼────┼─────────┼────■──┤ X ├
     └───┘┌─┴─┐┌─┴─┐       │    │  └───┘
q_5: ─────┤ X ├┤ X ├───────┼────┼────■──
          └───┘└───┘       │  ┌─┴─┐┌─┴─┐
q_6: ──────────────────────■──┤ X ├┤ X ├
                              └───┘└───┘
c: 7/═══════════════════════════════════
                                                                                                                    »
q_0: ─────────────────────────────────────■─────────■─────────■────■────■──»
                                          │         │         │  ┌─┴─┐  

In [19]:
from qrm_circuits import QRM_rec_punc_circuit
from qrm_utils import get_qiskit_circuit
from qrm_counts import rec_CX_count_punc
import tequila
r, m = 4, 5
c, i = QRM_rec_punc_circuit(r, m, state_prep=False)
d = get_qiskit_circuit(c)
from qrm_utils import draw_tequila_circuit
#print(tequila.simulate(c))
draw_tequila_circuit(c)

┌───┐                                                                 
 q_0: ──■────────────────────────────────────────────■────────────────────────■──────────────■──┤ X ├─────────────────────────────────────────────────────────────────
        │                                            │                        │            ┌─┴─┐└─┬─┘                                                                 
 q_1: ──┼─────────■──────────────────────────────────┼─────────■──────────────┼─────────■──┤ X ├──┼────■──────────────────────────────────────────────────────────────
        │         │                                  │         │              │         │  └───┘  │  ┌─┴─┐┌───┐                                                       
 q_2: ──┼────■────┼──────────────────────────────────┼────■────┼──────────────┼────■────┼─────────■──┤ X ├┤ X ├───────────────────────────────────────────────────────
        │    │    │                                  │    │    │            ┌─┴─┐  │    │            └───┘└─┬─┘                                                       
 q_3: ──┼────┼────┼─────────■────────────────────────┼────┼────┼─────────■──┤ X ├──┼────┼────■──────────────┼─────────■───────────────────────────────────────────────
        │    │    │         │                        │    │    │         │  └───┘┌─┴─┐  │    │              │       ┌─┴─┐                                             
 q_4: ──┼────┼────┼─────────┼────■───────────────────┼────┼────┼─────────┼────■──┤ X ├──┼────┼──────────────┼────■──┤ X ├─────────────────────────────────────────────
        │    │    │         │    │                   │    │    │         │    │  └───┘┌─┴─┐┌─┴─┐            │    │  └───┘                                             
 q_5: ──┼────┼────┼─────────┼────┼────■──────────────┼────┼────┼─────────┼────┼────■──┤ X ├┤ X ├────────────┼────┼────■───────────────────────────────────────────────
        │    │    │         │    │    │              │    │    │         │    │    │  └───┘└───┘            │  ┌─┴─┐┌─┴─┐┌───┐                                        
 q_6: ──┼────┼────┼────■────┼────┼────┼──────────────┼────┼────┼────■────┼────┼────┼────────────────────────■──┤ X ├┤ X ├┤ X ├────────────────────────────────────────
        │    │    │    │    │    │    │            ┌─┴─┐  │    │    │    │    │    │                           └───┘└───┘└─┬─┘                                        
 q_7: ──┼────┼────┼────┼────┼────┼────┼─────────■──┤ X ├──┼────┼────┼────┼────┼────┼────■──────────────■───────────────────┼──────────────■───────────────────────────
        │    │    │    │    │    │    │         │  └───┘┌─┴─┐  │    │    │    │    │    │              │                   │            ┌─┴─┐                         
 q_8: ──┼────┼────┼────┼────┼────┼────┼─────────┼────■──┤ X ├──┼────┼────┼────┼────┼────┼────■─────────┼───────────────────┼─────────■──┤ X ├─────────────────────────
        │    │    │    │    │    │    │         │    │  └───┘┌─┴─┐  │    │    │    │    │    │       ┌─┴─┐                 │         │  └───┘                         
 q_9: ──┼────┼────┼────┼────┼────┼────┼─────────┼────┼────■──┤ X ├──┼────┼────┼────┼────┼────┼────■──┤ X ├─────────────────┼─────────┼────■───────────────────────────
        │    │    │    │    │    │    │         │    │    │  └───┘┌─┴─┐  │    │    │    │    │    │  └───┘                 │       ┌─┴─┐┌─┴─┐                         
q_10: ──┼────┼────┼────┼────┼────┼────┼─────────┼────┼────┼────■──┤ X ├──┼────┼────┼────┼────┼────┼────────────────────────┼────■──┤ X ├┤ X ├─────────────────────────
        │    │    │    │    │    │    │         │    │    │    │  └───┘┌─┴─┐  │    │  ┌─┴─┐  │    │                        │    │  └───┘└───┘                         
q_11: ──┼────┼────┼────┼────┼────┼────┼─────────┼────┼────┼────┼────■──┤ X ├──┼────┼──┤ X ├──┼────┼────■───────────────────┼────┼─────────■───────────────────────────
        │    │    │    │    │    │    │         │    │    │    │    │  └───┘┌─┴─┐  │  └─

+0.3536|0000000> +0.3536|0111100> +0.3536|1011010> +0.3536|1100110> +0.3536|1101001> +0.3536|1010101> +0.3536|0110011> +0.3536|0001111> 

In [9]:
from qrm_circuits import QRM_rec_classical_circuit, QRM_rec_circuit
from qrm_utils import get_qiskit_circuit

c, i = QRM_rec_classical_circuit(2, 4)
print(get_qiskit_circuit(c))
c, i = QRM_rec_circuit(2, 4, only_cnots=True)
print(get_qiskit_circuit(c))

                                                                            
 q_0: ──■────────────────────────■────────────────────────■────■────────────
        │                        │                        │  ┌─┴─┐          
 q_1: ──┼────■───────────────────┼────■──────────────■────┼──┤ X ├──────────
        │    │                   │    │              │  ┌─┴─┐└───┘          
 q_2: ──┼────┼────■──────────────┼────┼─────────■────┼──┤ X ├──■────────────
        │    │    │              │    │         │  ┌─┴─┐└───┘┌─┴─┐          
 q_3: ──┼────┼────┼────■─────────┼────┼────■────┼──┤ X ├─────┤ X ├──────────
        │    │    │    │       ┌─┴─┐  │    │    │  └───┘     └───┘          
 q_4: ──┼────┼────┼────┼────■──┤ X ├──┼────┼────┼─────────■────■────────────
        │    │    │    │    │  └───┘┌─┴─┐  │    │         │  ┌─┴─┐          
 q_5: ──┼────┼────┼────┼────┼────■──┤ X ├──┼────┼────■────┼──┤ X ├──────────
        │    │    │    │    │    │  └───┘  │  ┌─┴─┐  │  ┌─┴─┐└───┘          

In [1]:
from qrm_circuits import QRM_rec_punc_circuit, QRM_rec_assym_punc_circuit
from qrm_utils import draw_tequila_circuit
c, i = QRM_rec_assym_punc_circuit(2, 6, 2, 6, state_prep=True, only_cnots=True)
draw_tequila_circuit(c)

q_0: ─────────────────────────────────────■────────────────────────■─────────────────────────────■─────────■───────────────────────────────────────────────────────────────────
                                           │                        │                             │         │                                                                   
 q_1: ─────────────────────────────────────┼────────────────────────┼─────────■───────────────────┼────■────┼────■────■─────────────────────────────────────────────────────────
                                           │                        │         │                   │    │  ┌─┴─┐  │  ┌─┴─┐                                                       
 q_2: ──■──────────────────────────────────┼────■───────────────────┼────■────┼─────────■─────────┼────┼──┤ X ├──┼──┤ X ├───────────────────────────────────────────────────────
        │                                  │    │                   │    │    │         │         │    │  └───┘  │  └───┘                                                       
 q_3: ──┼──────────────────────────────────┼────┼───────────────────┼────┼────┼────■────┼─────────┼────┼────■────┼─────────■────■───────────────────────────────────────────────
        │                                  │    │                   │    │    │    │    │       ┌─┴─┐  │    │    │         │  ┌─┴─┐                                             
 q_4: ──┼────■─────────────────────────────┼────┼────■──────────────┼────┼────┼────┼────┼────■──┤ X ├──┼────┼────┼────■────┼──┤ X ├─────────────────────────────────────────────
        │    │                             │    │    │              │    │    │    │    │    │  └───┘  │    │  ┌─┴─┐  │  ┌─┴─┐└───┘                                             
 q_5: ──┼────┼────■────────────────────────┼────┼────┼────■─────────┼────┼────┼────┼────┼────┼────■────┼────┼──┤ X ├──┼──┤ X ├──■───────────────────────────────────────────────
        │    │    │                        │    │    │    │         │    │    │    │  ┌─┴─┐  │    │    │    │  └───┘┌─┴─┐└───┘┌─┴─┐                                             
 q_6: ──┼────┼────┼────────────────────────┼────┼────┼────┼─────────┼────┼────┼────┼──┤ X ├──┼────┼────┼────┼───────┤ X ├─────┤ X ├─────────────────────────────────────────────
        │    │    │                        │    │    │    │         │    │    │    │  └───┘  │    │    │    │       └───┘     └───┘                                             
 q_7: ──┼────┼────┼────────────────────────┼────┼────┼────┼─────────┼────┼────┼────┼────■────┼────┼────┼────┼──────────────■─────────■────■─────────────────────────────────────
        │    │    │                        │    │    │    │       ┌─┴─┐  │    │    │    │    │    │    │    │              │         │  ┌─┴─┐                                   
 q_8: ──┼────┼────┼────■───────────────────┼────┼────┼────┼────■──┤ X ├──┼────┼────┼────┼────┼────┼────┼────┼────■─────────┼────■────┼──┤ X ├───────────────────────────────────
        │    │    │    │                   │    │    │    │    │  └───┘  │    │    │    │    │    │  ┌─┴─┐  │    │         │    │  ┌─┴─┐└───┘                                   
 q_9: ──┼────┼────┼────┼────■──────────────┼────┼────┼────┼────┼────■────┼────┼────┼────┼────┼────┼──┤ X ├──┼────┼────■────┼────┼──┤ X ├──■─────────────────────────────────────
        │    │    │    │    │              │    │    │    │    │    │  ┌─┴─┐  │    │    │    │    │  └───┘  │    │    │    │  ┌─┴─┐└───┘┌─┴─┐                                   
q_10: ──┼────┼────┼────┼────┼──────────────┼────┼────┼────┼────┼────┼──┤ X ├──┼────┼────┼────┼────┼────■────┼────┼────┼────┼──┤ X ├─────┤ X ├───────────────────────────────────
        │    │    │    │    │              │    │    │    │    │    │  └───┘  │    │    │    │    │    │  ┌─┴─┐  │    │  ┌─┴─┐└───┘     └───┘                                   
q_11: ──┼────┼────┼────┼────┼────■─────────┼────┼────┼────┼────┼────┼────■────┼────┼────┼────┼────┼────┼──┤

      ┌───┐                                                                 
 q_0: ┤ H ├─────────────────────────────────────■───────────────────■────■──
      ├───┤                                     │                   │  ┌─┴─┐
 q_1: ┤ H ├─────────────────■───────────────────┼────■─────────■────┼──┤ X ├
      ├───┤                 │                   │    │         │  ┌─┴─┐└───┘
 q_2: ┤ H ├─────────────────┼────■──────────────┼────┼────■────┼──┤ X ├──■──
      └───┘                 │    │              │    │    │  ┌─┴─┐└───┘┌─┴─┐
 q_3: ───────■──────────────┼────┼────■─────────┼────┼────┼──┤ X ├─────┤ X ├
      ┌───┐  │              │    │    │       ┌─┴─┐  │    │  └───┘     └───┘
 q_4: ┤ H ├──┼──────────────┼────┼────┼────■──┤ X ├──┼────┼─────────■────■──
      └───┘  │              │    │    │    │  └───┘┌─┴─┐  │         │  ┌─┴─┐
 q_5: ───────┼────■─────────┼────┼────┼────┼───────┤ X ├──┼────■────┼──┤ X ├
             │    │         │    │    │    │       └───┘┌─┴─┐  │  ┌─┴─┐└───┘
 q_6: ───────┼────┼────■────┼────┼────┼────┼────────────┤ X ├──┼──┤ X ├──■──
             │    │    │    │    │  ┌─┴─┐  │            └───┘┌─┴─┐└───┘┌─┴─┐
 q_7: ───────┼────┼────┼────┼────┼──┤ X ├──┼─────────────────┤ X ├─────┤ X ├
      ┌───┐  │    │    │    │    │  └───┘  │                 └───┘     └───┘
 q_8: ┤ H ├──┼────┼────┼────┼────┼────■────┼──────────────■──────────────■──
      └───┘  │    │    │  ┌─┴─┐  │    │    │              │            ┌─┴─┐
 q_9: ───────┼────┼────┼──┤ X ├──┼────┼────┼────■─────────┼─────────■──┤ X ├
             │    │    │  └───┘┌─┴─┐┌─┴─┐  │    │         │         │  └───┘
q_10: ───────┼────┼────┼───────┤ X ├┤ X ├──┼────┼────■────┼─────────┼────■──
           ┌─┴─┐  │    │       └───┘└───┘  │  ┌─┴─┐  │    │         │  ┌─┴─┐
q_11: ─────┤ X ├──┼────┼───────────────────┼──┤ X ├──┼────┼────■────┼──┤ X ├
           └───┘  │    │                 ┌─┴─┐└───┘  │  ┌─┴─┐  │    │  └───┘
q_12: ────────────┼────┼─────────────────┤ X ├──■────┼──┤ X ├──┼────┼────■──
                ┌─┴─┐  │                 └───┘  │    │  └───┘  │  ┌─┴─┐┌─┴─┐
q_13: ──────────┤ X ├──┼────■───────────────────┼────┼─────────┼──┤ X ├┤ X ├
                └───┘┌─┴─┐  │                 ┌─┴─┐┌─┴─┐       │  └───┘└───┘
q_14: ───────────────┤ X ├──┼─────────────────┤ X ├┤ X ├───────┼────■───────
                     └───┘┌─┴─┐               └───┘└───┘     ┌─┴─┐┌─┴─┐     
q_15: ────────────────────┤ X ├──────────────────────────────┤ X ├┤ X ├─────
                          └───┘                              └───┘└───┘     

In [18]:
from qrm_circuits import add_punc_entanglers

add_punc_entanglers(2, 2, 3)

circuit: 
X(target=(10,), control=(2,))
X(target=(12,), control=(4,))
X(target=(13,), control=(5,))

In [22]:
from qrm_matrices import Grm, get_QRM_generators_r1r2

G = Grm(2, 3)
get_QRM_generators_r1r2(2, 2, 3)


[array([0, 0, 0, 1, 0, 0, 0, 1]),
 array([0, 0, 0, 0, 0, 1, 0, 1]),
 array([0, 0, 0, 0, 0, 0, 1, 1])]

In [10]:
from qrm_matrices import Grm

G = Grm(2, 4)
for a in G:
    a_s = '{}'.format(a[0])
    for i, b in enumerate(a[1:]):
        if (i+1) % 4 == 0:
            a_s += ' & '
        a_s += '{}'.format(b)
    a_s += '\\\\'
    print(a_s)

1111 & 1111 & 1111 & 1111\\
0101 & 0101 & 0101 & 0101\\
0011 & 0011 & 0011 & 0011\\
0001 & 0001 & 0001 & 0001\\
0000 & 1111 & 0000 & 1111\\
0000 & 0101 & 0000 & 0101\\
0000 & 0011 & 0000 & 0011\\
0000 & 0000 & 1111 & 1111\\
0000 & 0000 & 0101 & 0101\\
0000 & 0000 & 0011 & 0011\\
0000 & 0000 & 0000 & 1111\\


In [16]:
from qrm_utils import get_int_set

get_int_set(3, 2, reversed=False)

[0, 1]

In [5]:
from qrm_utils import *
def add_ind_to_key_ind(dict_, m, to_add = [0]):
        
    dict_new = {}
    for k, v in zip(dict_.keys(), dict_.values()):
        k_new = get_set_int(add_to_set(get_int_set(k, m = m, reversed=True), [0]))
        dict_new[k_new] = v
    return dict_new

add_ind_to_key_ind({0: 2, 1: 3}, 4, to_add=[0])

{1: 2, 3: 3}

In [10]:
get_int_set(5, 4, reversed=False)

[1, 3]

In [5]:
print(tequila.simulate(c),'\n', tequila.simulate(c2))

+0.3536|0000000> +0.3536|0111100> +0.3536|1011010> +0.3536|1100110> +0.3536|1101001> +0.3536|1010101> +0.3536|0110011> +0.3536|0001111>  
 +0.3536|0000000> +0.3536|0111100> +0.3536|1101010> +0.3536|1010110> +0.3536|1011001> +0.3536|1100101> +0.3536|0110011> +0.3536|0001111> 


In [5]:
from qrm_circuits import QRM_rec_circuit, QRM_rec_assym_circuit
from qrm_utils import get_qiskit_circuit
from qrm_counts import rec_CX_count_assym, rec_CX_count
import tequila
r, m = 2, 4
c, i = QRM_rec_circuit(r, m, only_cnots=True)
print(get_qiskit_circuit(c))
print(len(c.gates))
print(rec_CX_count(r, m))
print(tequila.simulate(c))

                                                                       
 q_0: ──────────────────────■────────────────────────■────■────────────
                            │                        │  ┌─┴─┐          
 q_1: ──■───────────────────┼────■──────────────■────┼──┤ X ├──────────
        │                   │    │              │  ┌─┴─┐└───┘          
 q_2: ──┼────■──────────────┼────┼─────────■────┼──┤ X ├──■────────────
        │    │              │    │         │  ┌─┴─┐└───┘┌─┴─┐          
 q_3: ──┼────┼────■─────────┼────┼────■────┼──┤ X ├─────┤ X ├──────────
        │    │    │       ┌─┴─┐  │    │    │  └───┘     └───┘          
 q_4: ──┼────┼────┼────■──┤ X ├──┼────┼────┼─────────■────■────────────
        │    │    │    │  └───┘┌─┴─┐  │    │         │  ┌─┴─┐          
 q_5: ──┼────┼────┼────┼────■──┤ X ├──┼────┼────■────┼──┤ X ├──────────
        │    │    │    │    │  └───┘  │  ┌─┴─┐  │  ┌─┴─┐└───┘          
 q_6: ──┼────┼────┼────┼────┼────■────┼──┤ X ├──┼──┤ X ├──■─────

In [5]:
from qrm_circuits import QRM_rec_classical_circuit, QRM_rec_circuit
from qrm_utils import get_qiskit_circuit

c, i = QRM_rec_classical_circuit(1, 3)
print(get_qiskit_circuit(c))
print(len(c.gates))

                                   
q_0: ──■───────────────────■────■──
       │                   │  ┌─┴─┐
q_1: ──┼────■─────────■────┼──┤ X ├
       │    │         │  ┌─┴─┐└───┘
q_2: ──┼────┼────■────┼──┤ X ├──■──
       │    │    │  ┌─┴─┐└───┘┌─┴─┐
q_3: ──┼────┼────┼──┤ X ├─────┤ X ├
     ┌─┴─┐  │    │  └───┘     └───┘
q_4: ┤ X ├──┼────┼────■─────────■──
     └───┘┌─┴─┐  │    │       ┌─┴─┐
q_5: ─────┤ X ├──┼────┼────■──┤ X ├
          └───┘┌─┴─┐┌─┴─┐  │  └───┘
q_6: ──────────┤ X ├┤ X ├──┼────■──
               └───┘└───┘┌─┴─┐┌─┴─┐
q_7: ────────────────────┤ X ├┤ X ├
                         └───┘└───┘
c: 8/══════════════════════════════
                                   
11


In [2]:
from qrm_matrices import get_QRM_generator
from qrm_utils import *

def get_QRM_punc_generator(C1_params: tuple, C2_params: tuple):
    G2perp, G1q = get_QRM_generator(C1_params=C1_params, C2_params=C2_params)
    G2perpt = puncture_matrix(G2perp)
    G1qp = puncture_matrix(G1q)
    print(G2perpt)
    G2perpp = []
    for row in G2perpt:
        if np.sum(row) != len(row):
            G2perpp.append(row)
        else:
            G1qp.append(row)
    return (G2perpp, G1qp)

get_QRM_punc_generator((2, 4), (2, 4))

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1], [0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]]


([[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
  [0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
  [0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1],
  [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]],
 [[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
  [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
  [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [4]:
#CSS encoders
import tequila
from qrm_circuits import QRM_std_circuit, QRM_rec_circuit
from qrm_utils import *
from qrm_circuits import QRM_std_circuit
from qrm_matrices import *

#need to generalize stuff
r, m = 2, 4
circuit, info = QRM_rec_circuit(r, m, partitions=[1, [1], [1]])
std_circuit, info2 = QRM_std_circuit(r, m)
qiskit_cir = get_qiskit_circuit(circuit)
print(info, qiskit_cir, circuit.depth, len(circuit.gates))
check_if_same_circuit(circuit, std_circuit)

None       ┌───┐                                                                 
 q_0: ┤ H ├─────────────────────────────────────■───────────────────■────■──
      ├───┤                                     │                   │  ┌─┴─┐
 q_1: ┤ H ├─────────────────■───────────────────┼────■─────────■────┼──┤ X ├
      ├───┤                 │                   │    │         │  ┌─┴─┐└───┘
 q_2: ┤ H ├─────────────────┼────■──────────────┼────┼────■────┼──┤ X ├──■──
      └───┘                 │    │              │    │    │  ┌─┴─┐└───┘┌─┴─┐
 q_3: ───────■──────────────┼────┼────■─────────┼────┼────┼──┤ X ├─────┤ X ├
      ┌───┐  │              │    │    │       ┌─┴─┐  │    │  └───┘     └───┘
 q_4: ┤ H ├──┼──────────────┼────┼────┼────■──┤ X ├──┼────┼─────────■────■──
      └───┘  │              │    │    │    │  └───┘┌─┴─┐  │         │  ┌─┴─┐
 q_5: ───────┼────■─────────┼────┼────┼────┼───────┤ X ├──┼────■────┼──┤ X ├
             │    │         │    │    │    │       └───┘┌─┴─┐  │  ┌─┴─┐

True

In [1]:
import tequila
from qrm_circuits import QRM_rec_circuit, QRM_std_circuit
only_cnots = False
circuit, info1 = QRM_rec_circuit(3, 4, only_cnots=only_cnots)
circuit_std, info2 = QRM_std_circuit(3, 4, only_cnots=only_cnots, transform_rows=False)
msg = tequila.QCircuit()

msg += tequila.gates.H(target=info1[0][0])
msg += tequila.gates.CX(target=info1[0][1], control=info1[0][0])
print(len(circuit_std.gates), len(circuit.gates))
from qrm_utils import check_if_same_circuit

check_if_same_circuit(msg + circuit, msg + circuit_std)

66 33


True

In [4]:
#generate check matrix and find the diagonalizing circuit
from qrm_matrices import *
r, m = (2, 3)
G = Grm(r, m)
H = Hrm(r, m)
#HCSS = 

import tequila
from tequila.grouping.binary_rep import BinaryPauliString, BinaryHamiltonian

H = Hqrm(1, 3)

bps = [BinaryPauliString(binary_vector=bv, coeff=1.0) for bv in H]
BH = BinaryHamiltonian(binary_terms=bps)

stuff = BH.get_qubit_wise()

ImportError: attempted relative import with no known parent package